In [ ]:
#test

# 必要なライブラリのインポート
from bs4 import BeautifulSoup
import requests
import time
import re

# スクレイピングするURLのリスト
urls = [
    "https://nf3.sakura.ne.jp/php/stat_disp/stat_disp.php?y=2022&leg=0&mon=0&tm=DB&vst=all",
    "https://nf3.sakura.ne.jp/php/stat_disp/stat_disp.php?y=0&leg=0&mon=0&tm=DB&vst=all"
]

# スクレイピングしたデータを保存するためのリスト
scraped_data = []

for url in urls:
    # Webページからコンテンツを取得
    response = requests.get(url)
    html_soup = BeautifulSoup(response.content, 'html.parser')

    # ここでHTMLから必要なデータを抽出するコードを書く
    date = html_soup.find_all('tr', class_='Index')

    for p in date:
      time.sleep(1)
      dates = p.find_all('th')

      if len(dates) > 2:  # 少なくとも2つの列があることを確認
        # 0列目、1列目のテキストを抽出
        date0 = dates[0].text.strip()
        date1 = dates[1].text.strip()

    th = html_soup.find_all('tr', class_='Index2')

    for t in th:
      time.sleep(1)
      heads = t.find_all('th')

    if len(heads) > 7:
        # 少なくとも7つの列があることを確認
        head0 = heads[0].text.strip()
        head5 = heads[5].text.strip()
        head15 = heads[15].text.strip()
        head16 = heads[16].text.strip()
        head18 = heads[18].text.strip()
        print(date0, date1, head0, head5, head15, head16, head18)

    n = html_soup.find_all('tr',onmouseover="M_over(this)")
    for i in n:
      time.sleep(1)
      cols = i.find_all('td')
      if len(cols) > 7:  # 少なくとも7つの列があることを確認
        # 0列目、1列目、3列目のテキストを抽出
        col0 = cols[0].text.strip()
        col1 = cols[1].text.strip()
        col2 = cols[2].text.strip()
        col7 = cols[7].text.strip()
        col17 = cols[17].text.strip()
        col18 = cols[18].text.strip()
        col20 = cols[20].text.strip()
        
        print(col0, col1, col2, col7, col17, col18, col20)

import sqlite3

# データベースファイルに接続
conn = sqlite3.connect('scraped_data.db')
cursor = conn.cursor()

# ヘッダー情報を保存するテーブル
cursor.execute('''
    CREATE TABLE IF NOT EXISTS headers (
        date0 TEXT,
        date1 TEXT,
        head0 TEXT,
        head5 TEXT,
        head15 TEXT,
        head16 TEXT,
        head18 TEXT
    )
''')

# データ行を保存するテーブル
cursor.execute('''
    CREATE TABLE IF NOT EXISTS data (
        col0 TEXT,
        col1 TEXT,
        col2 TEXT,
        col7 TEXT,
        col17 TEXT,
        col18 TEXT,
        col20 TEXT
    )
''')

# コミットして変更を保存
conn.commit()

# ヘッダー情報の挿入
cursor.execute('''
    INSERT INTO headers (date0, date1, head0, head5, head15, head16, head18)
    VALUES (?, ?, ?, ?, ?, ?, ?)
''', (date0, date1, head0, head5, head15, head16, head18))

# データ行の挿入
# スクレイピングしたデータ行をリストに保存
data_rows = []
for i in n:
      time.sleep(1)
      cols = i.find_all('td')
      if len(cols) > 7:  # 少なくとも7つの列があることを確認
        # 0列目、1列目、3列目のテキストを抽出
        col0 = cols[0].text.strip()
        col1 = cols[1].text.strip()
        col2 = cols[2].text.strip()
        col7 = cols[7].text.strip()
        col17 = cols[17].text.strip()
        col18 = cols[18].text.strip()
        col20 = cols[20].text.strip()

        data_rows.append((col0, col1, col2, col7, col17, col18, col20))

# データベースにデータ行を挿入
cursor.executemany('''
    INSERT INTO data (col0, col1, col2, col7, col17, col18, col20)
    VALUES (?, ?, ?, ?, ?, ?, ?)
''', data_rows)

# コミットして変更を保存
conn.commit()

# データベース接続を閉じる
conn.close()